In [1]:
# Limitasi Synchronous:
# ---------------------------------
# Audio Maximum 1 Menit
# Audio Maximum 10 MB
# ---------------------------------

# Import Library
from google.cloud.speech_v2 import SpeechClient
from google.cloud.speech_v2.types import cloud_speech
from google.oauth2 import service_account


# Function transkrip audio Synchronous
def transcribe_file_v2(project_id: str, audio_file: str) -> cloud_speech.RecognizeResponse:
    # Instantiates client
    credentials = service_account.Credentials.from_service_account_file('gcloud_apikey.json')
    client = SpeechClient(credentials=credentials)

    # Baca file dalam bytes
    with open(audio_file, "rb") as f:
        content = f.read()

    # Config untuk speech recognition
    config = cloud_speech.RecognitionConfig(
        auto_decoding_config=cloud_speech.AutoDetectDecodingConfig(),
        language_codes=["en-US"],
        model="long",
    )

    # Membentuk objek request
    request = cloud_speech.RecognizeRequest(
        recognizer=f"projects/{project_id}/locations/global/recognizers/_",
        config=config,
        content=content,
    )

    # Transkrip audio menjadi teks
    response = client.recognize(request=request)

    # Print respon transkrip
    for result in response.results:
        print(f"Transcript: {result.alternatives[0].transcript}")

    # Return respon transkrip
    return response

In [2]:
%%HTML
<video width="500" controls>
    <source src="speech_data/Anime.mp4" type="video/mp4">
    <track src="Anime.srt" kind="subtitles">
</video>

In [3]:
# Import Library
from moviepy.editor import VideoFileClip

# Deklarasi input dan output
mp4_file = "speech_data/Anime.mp4"
mp3_file = "speech_data/Anime.mp3"

# Load video
video_clip = VideoFileClip(mp4_file)

# Ekstrak audio dari video
audio_clip = video_clip.audio

# Simpan audio ke file
audio_clip.write_audiofile(mp3_file)

# Menutup file audio dan video
audio_clip.close()
video_clip.close()

MoviePy - Writing audio in speech_data/Anime.mp3


MoviePy - Done.


In [5]:
## Menyimpan Hasil Transkrip
hasil_transkrip = transcribe_file_v2(
    project_id='data-science-programming-ti24',
    audio_file='speech_data/Anime.mp3')

Transcript: she know that I I just wanted to cheer you see we you and I are the same I can't say the things that I truly feel just stop it and you're wrong we're nothing alike
Transcript:  it's really embarrassed me
Transcript:  have you considered my feelings at all everyone just hurt this I seriously hate pushy people like you
Transcript:  what if I told you in person just leave me alone didn't you hear me
Transcript:  but I hate you
Transcript:  I see
Transcript:  you hate me that much okay well then
Transcript:  hey


In [6]:
print(hasil_transkrip)

results {
  alternatives {
    transcript: "she know that I I just wanted to cheer you see we you and I are the same I can\'t say the things that I truly feel just stop it and you\'re wrong we\'re nothing alike"
    confidence: 0.877077341
  }
  result_end_offset {
    seconds: 13
    nanos: 880000000
  }
  language_code: "en-US"
}
results {
  alternatives {
    transcript: " it\'s really embarrassed me"
    confidence: 0.944916487
  }
  result_end_offset {
    seconds: 17
    nanos: 140000000
  }
  language_code: "en-US"
}
results {
  alternatives {
    transcript: " have you considered my feelings at all everyone just hurt this I seriously hate pushy people like you"
    confidence: 0.959103167
  }
  result_end_offset {
    seconds: 25
    nanos: 920000000
  }
  language_code: "en-US"
}
results {
  alternatives {
    transcript: " what if I told you in person just leave me alone didn\'t you hear me"
    confidence: 0.890649498
  }
  result_end_offset {
    seconds: 31
    nanos: 4600

In [7]:
list_hasil_transkrip = list()

for hasil in hasil_transkrip.results:
    teks = f'{hasil.alternatives[0].transcript}'
    list_hasil_transkrip.append(teks)

list_hasil_transkrip

["she know that I I just wanted to cheer you see we you and I are the same I can't say the things that I truly feel just stop it and you're wrong we're nothing alike",
 " it's really embarrassed me",
 ' have you considered my feelings at all everyone just hurt this I seriously hate pushy people like you',
 " what if I told you in person just leave me alone didn't you hear me",
 ' but I hate you',
 ' I see',
 ' you hate me that much okay well then',
 ' hey']

# PREDICT

In [8]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import TextVectorization #tokenization|
import pickle

In [9]:
model = tf.keras.models.load_model('toxic-v1.h5')

In [10]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# toxic == toxic
# sever_toxic == toxic_parah
# obscene == cabul
# threat == ancaman
# insult == menyinggung 
# indentity_hate == benci personal

In [11]:
with open('vectorizer_config.pkl', 'rb') as f:
    vectorizer_config = pickle.load(f)
with open('vectorizer_vocab.pkl', 'rb') as f:
    vectorizer_vocab = pickle.load(f)

In [12]:
vectorizer = TextVectorization.from_config(vectorizer_config)
vectorizer.set_vocabulary(vectorizer_vocab)

In [13]:
# df = pd.read_csv(os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv', 'train.csv'))
# X = df['comment_text'].values

In [14]:
# df[df['threat']==1]

In [15]:
# MAX_FEATURES = 200000
# vectorizer = TextVectorization(max_tokens=MAX_FEATURES, output_sequence_length=1800, output_mode='int')
# vectorizer.adapt(X)

In [16]:
# vectorizer.set_vocabulary(vocab)

In [17]:
input_data = list_hasil_transkrip

In [18]:
# Vectorize each input text in the list
vectorized_texts = [vectorizer(text) for text in input_data]

# Pad the sequences to the same length
padded_texts = tf.keras.preprocessing.sequence.pad_sequences(vectorized_texts, maxlen=1800)

In [ ]:
predictions = model.predict(padded_texts)

In [ ]:
binary_predictions = (predictions > 0.5).astype(int)
binary_predictions

In [ ]:
# Print predictions
for i, text in enumerate(input_data):
    print("Text:", text)
    for label, pred in zip(labels, binary_predictions[i]):
        print(f"{label}: {pred}")
    print()

In [ ]:
total_predictions = binary_predictions.sum(axis=0)
total_predictions

In [ ]:
for i, (label, total) in enumerate(zip(labels, total_predictions)):
    print(f"{label}: {total}")

In [ ]:
label_index = {label: [] for label in labels}

total_predictions = binary_predictions.sum(axis=0)

for i, (prediction, text) in enumerate(zip(binary_predictions, input_data)):
    # print(f"Text: {text}")
    for j, (label, pred) in enumerate(zip(labels, prediction)):
        if pred == 1:
            print(f"Text: {text}")
            print(f"Prediction: {label} Value: {predictions[i][j]}")
            label_index[label].append(i)  # Store the index where the label was predicted as 1
    print()

for label, total in zip(labels, total_predictions):
    print(f"Total {label} predictions: {total}")
    print(f"Index Kalimat yang terdeteksi {label}: {label_index[label]}")
    print()

In [55]:
input_str = vectorizer(coba_list[0])

In [56]:
prediction = model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 1s 547ms/step


In [57]:
prediction

array([[0.8127243 , 0.03915123, 0.4758387 , 0.04838365, 0.45078787,
        0.08074123]], dtype=float32)

In [58]:
binary_prediction = (prediction > 0.5).astype(int)
binary_prediction

array([[1, 0, 0, 0, 0, 0]])

In [59]:
for label, pred in zip(labels, binary_prediction[0]):
    print(f"{label}: {pred}")

toxic: 1
severe_toxic: 0
obscene: 0
threat: 0
insult: 0
identity_hate: 0


In [60]:
for label, pred in zip(labels, binary_prediction[0]):
    if pred == 1:
        print(label)

toxic
